In [1]:
import numpy as np
import pandas as pd
import pymysql as mysql
import matplotlib.pyplot as plt

In [8]:
import requests

url = "https://air-quality-by-api-ninjas.p.rapidapi.com/v1/airquality"

querystring = {"city":"San Diego"}

headers = {
	"X-RapidAPI-Key": "570296f95emsh5f009ea13d4ed2cp106b38jsnbd81b0645b00",
	"X-RapidAPI-Host": "air-quality-by-api-ninjas.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"CO": {"concentration": 267.03, "aqi": 3}, "NO2": {"concentration": 15.94, "aqi": 19}, "O3": {"concentration": 62.94, "aqi": 62}, "SO2": {"concentration": 0.73, "aqi": 1}, "PM2.5": {"concentration": 5.4, "aqi": 17}, "PM10": {"concentration": 10.91, "aqi": 10}, "overall_aqi": 62}
